### Imports and Commonly Used Symbols

In [1]:
import gzip
import pickle
import os
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
from pylab import rcParams
import pandas as pd
import pystan
from matplotlib.ticker import MultipleLocator
from scipy.stats import gaussian_kde
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
rcParams['figure.figsize'] = 16, 10
COMMON_SEED = 1234

def check_convergence(fit, also_print=False):
    report = print if also_print else lambda x: None
    
    def all_rhat_small_enough(fit):
        return all(dict(fit.summary())['summary'][:, -1] < 1.1)
    
    def max_treedepth_exceeded(fit, max_depth = 10):
        """Check transitions that ended prematurely due to maximum tree depth limit"""
        sampler_params = fit.get_sampler_params(inc_warmup=False)
        depths = [x for y in sampler_params for x in y['treedepth__']]
        n = sum(1 for x in depths if x == max_depth)
        if n > 0:
            report('Run again with max_depth set to a larger value to avoid saturation')        
        N = len(depths)
        report(('{} of {} iterations saturated the maximum tree depth of {}'
               + ' ({}%)').format(n, N, max_depth, 100 * n / N))
        return float(n) / N
    
    def e_bfmi_all_low_enough(fit):
        """
        Checks the energy Bayesian fraction of missing information (E-BFMI).
        E-BFMI below 0.2 indicates you may need to reparameterize your model
        """
        sampler_params = fit.get_sampler_params(inc_warmup=False)
        for chain_num, s in enumerate(sampler_params):
            energies = s['energy__']
            numer = sum((energies[i] - energies[i - 1])**2 for i in range(1, len(energies))) / len(energies)
            denom = np.var(energies)
            if numer / denom < 0.2:
                report('Chain {}: E-BFMI = {}'.format(chain_num, numer / denom))
                report('E-BFMI below 0.2 indicates you may need to reparameterize your model')
                return False
        return True

    def fraction_of_transitions_which_ended_with_divergence(fit):
        """Check transitions that ended with a divergence"""
        sampler_params = fit.get_sampler_params(inc_warmup=False)
        divergent = [x for y in sampler_params for x in y['divergent__']]
        n = sum(divergent)
        N = len(divergent)
        report('{} of {} iterations ended with a divergence ({}%)'
              .format(n, N, 100 * n / N))
        if n > 0:
            report('Try running with larger adapt_delta to remove the divergences')
        return n / N
    
    print("all_rhat_small_enough:", all_rhat_small_enough(fit))
    print("max_treedepth_exceeded:", max_treedepth_exceeded(fit) < 0.02)
    print("e_bfmi_all_low_enough", e_bfmi_all_low_enough(fit))
    print("fraction_of_transitions_which_ended_with_divergence", \
          fraction_of_transitions_which_ended_with_divergence(fit) <= 5E-3)
#     report("##### All convergence checks passed successfully. #####")

In [3]:
births_2000s_df = pd.read_csv('US_births_2000-2014_SSA.csv')
births_2000s_df_with_index = births_2000s_df\
    .rename(columns={'date_of_month': 'day'})\
    .set_index(pd.to_datetime(
        births_2000s_df.rename(columns={'date_of_month': 'day'})
        [['year', 'month', 'day']]))\
    .assign(weekday_name=lambda df: df.index.weekday_name)\
    .assign(day_of_year=lambda df: df.index.dayofyear)\
    .assign(week_of_year=lambda df: df.index.weekofyear)
births_2000s_df_with_index.tail()

,year,month,day,day_of_week,births,weekday_name,day_of_year,week_of_year
2014-12-27,2014,12,27,6,8656,Saturday,361,52
2014-12-28,2014,12,28,7,7724,Sunday,362,52
2014-12-29,2014,12,29,1,12811,Monday,363,1
2014-12-30,2014,12,30,2,13634,Tuesday,364,1
2014-12-31,2014,12,31,3,11990,Wednesday,365,1


In [4]:
def weekday_priors_by_year_2000():
    df_2000 = births_2000s_df_with_index[lambda df: df.year == 2000]
    return df_2000.groupby('day_of_week').agg(['mean', 'std']).births

weekday_priors_by_year_2000()

,mean,std
day_of_week,,
1,11514.461538,1036.715902
2,12870.846154,790.289954
3,12762.269231,422.905919
4,12735.461538,813.567006
5,12524.500000,634.105530
6,9049.830189,344.643056
7,8014.433962,310.302366


In [5]:
def get_train_and_test_for_sliding_window(test_date):
    num_data_points_before_first_test_date = datetime(2014, 1, 1) - datetime(2001, 1, 1)
    df = births_2000s_df_with_index[['day_of_week', 'births']]\
        [lambda df: df.index <= test_date + timedelta(days=6)]\
        [lambda df: df.index >= test_date - num_data_points_before_first_test_date]
    return {
        'x_train': df.day_of_week[:-7].values,
        'y_train': df.births[:-7].values,
        'x_test': df.day_of_week[-7:].values,
        'y_test': df.births[-7:].values
    }
        
get_train_and_test_for_sliding_window(datetime(2014, 1, 1))

{'x_test': array([3, 4, 5, 6, 7, 1, 2]),
 'x_train': array([1, 2, 3, ..., 7, 1, 2]),
 'y_test': array([ 8018, 11171, 12317,  8199,  7174, 11400, 12310]),
 'y_train': array([ 7663, 10635, 12449, ...,  7896, 13096, 12525])}

In [6]:
def rmse(prediction_errors):
    return (prediction_errors ** 2).mean() ** 0.5

In [7]:
def fit_stan_model_on_sliding_windows(iterations, stan_data_extractor, persist_path, stan_model, num_windows):
    if not os.path.exists(persist_path):
        os.mkdir(persist_path)
    all_prediction_errors = []
    for i in range(num_windows):
        data = get_train_and_test_for_sliding_window(datetime(2014, 1, 1) + timedelta(days=i * 7))
        fit = stan_model.sampling(seed=COMMON_SEED, data=stan_data_extractor(data), iter=iterations, chains=1,
                                 control=dict(adapt_delta=0.99))  # max_treedepth=20
        print(fit)
        check_convergence(fit, also_print=True)
        pred_err = fit.extract()['y_pred'].mean(axis=0) - data['y_test']
        all_prediction_errors.append(pred_err)
        with gzip.open(os.path.join(persist_path, 'fit%d_summary.pkl.gz' % i), 'wb') as f:
            pickle.dump(fit.summary(), f)
        with gzip.open(os.path.join(persist_path, 'fit%d_extract.pkl.gz' % i), 'wb') as f:
            pickle.dump(fit.extract(), f)         
        with gzip.open(os.path.join(persist_path, 'fit%d_pred_err.pkl.gz' % i), 'wb') as f:
            pickle.dump(pred_err, f)
        with gzip.open(os.path.join(persist_path, 'fit%d_y_test.pkl.gz' % i), 'wb') as f:
            pickle.dump(data['y_test'], f)
        with gzip.open(os.path.join(persist_path, 'fit%d.pkl.gz' % i), 'wb') as f:
            pickle.dump(fit, f)                        
    return rmse(np.concatenate(all_prediction_errors))

In [8]:
def priors_for_weekend_vs_workday_by_year_2000():
    df_2000 = births_2000s_df_with_index[lambda df: df.year == 2000]
    return df_2000\
        .assign(is_weekend=lambda df: df.day_of_week.isin([6, 7]))\
        .groupby('is_weekend')\
        .agg(['mean', 'std'])\
        .births

priors_for_weekend_vs_workday_by_year_2000()
# priors_for_weekend_vs_workday_by_year_2000()[lambda df: df.index == True]['mean'].values[0]

,mean,std
is_weekend,,
False,12481.507692,909.221295
True,8532.132075,614.062616


In [20]:
model_even_simpler_hier_negbin = pystan.StanModel(model_code='''
data {
    int<lower=0> T; // Number of predictions.
    int<lower=0> N; // Number of data points.
    int y_train[N]; // Train data points.
    int y_test[T]; // Test data points, for log-likelihood.
    int<lower=1,upper=7> x_train[N]; // Weekday indicator for each observed data point.
    int<lower=1,upper=7> x_test[T]; // Weekday indicator for each prediction.
}
parameters {
    real<lower=1> phi[7]; // Prevent too low phi, which yields enormous variance in NegBin.
    real<lower=0> mu[7];
    real<lower=0> phi_weekend;
    real<lower=0> phi_workday;    
    real<lower=0> mu_weekend;
    real<lower=0> mu_workday;
    real<lower=0> sigma_mu_weekend;
    real<lower=0> sigma_mu_workday;
    real<lower=0> sigma_phi_weekend;
    real<lower=0> sigma_phi_workday;
}
model {
    mu_weekend ~ normal(8532, 10);
    mu_workday ~ normal(12482, 10);
    sigma_mu_weekend ~ cauchy(614, 10);
    sigma_mu_workday ~ cauchy(909, 10);
    mu[6] ~ normal(mu_weekend, sigma_mu_weekend);
    mu[7] ~ normal(mu_weekend, sigma_mu_weekend);
    for (i in 1:5) {
        mu[i] ~ normal(mu_workday, sigma_mu_workday);
    }
    
    phi_weekend ~ normal(200, 10);
    sigma_phi_weekend ~ cauchy(0, 10);
    
    phi[6] ~ normal(phi_weekend, sigma_phi_weekend);
    phi[7] ~ normal(phi_weekend, sigma_phi_weekend);
    
    phi_workday ~ normal(200, 10);
    sigma_phi_workday ~ cauchy(0, 10);    
    for (i in 1:5) {
        phi[i] ~ normal(phi_workday, sigma_phi_workday);
    }    
    
    y_train ~ neg_binomial_2(mu[x_train], phi[x_train]);
}
generated quantities {
    real loglik;
    vector[T] y_pred;
    loglik = 0;
    for (t in 1:T) {
        y_pred[t] = neg_binomial_2_rng(mu[x_test[t]], phi[x_test[t]]);
        loglik += neg_binomial_2_lpmf(y_test[t] | mu[x_test[t]], phi[x_test[t]]);
    }
}
''')

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_2e1ff531a6913631bda4622f8ce9a981 NOW.


In [21]:
with open('model_even_simpler_hier_negbin.pkl', 'wb') as f:
    pickle.dump(model_even_simpler_hier_negbin, f)

In [22]:
model_even_simpler_hier_negbin = pickle.load(open('model_even_simpler_hier_negbin.pkl', 'rb'))

In [23]:
fit_stan_model_on_sliding_windows(
    100,
    lambda data: dict(
        x_train=data['x_train'],
        x_test=data['x_test'],
        y_train=data['y_train'],
        y_test=data['y_test'],
        T=7,
        N=len(data['x_train'])),
    'fit_even_simpler_hier_negbin', model_even_simpler_hier_negbin, 1)

Inference for Stan model: anon_model_2e1ff531a6913631bda4622f8ce9a981.
1 chains, each with iter=100; warmup=50; thin=1; 
post-warmup draws per chain=50, total post-warmup draws=50.

                    mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
phi[0]              1.16  4.9e-3 8.4e-3   1.14   1.15   1.16   1.16   1.16      3   1.68
phi[1]              1.17  3.2e-3 6.5e-3   1.16   1.17   1.18   1.18   1.18      4   1.21
phi[2]              1.17  8.2e-3   0.02   1.13   1.17   1.18   1.18   1.18      4   1.32
phi[3]              1.06  4.4e-4 1.1e-3   1.06   1.06   1.06   1.06   1.06      6   1.13
phi[4]              1.19  3.6e-3 7.1e-3   1.17   1.19   1.19    1.2    1.2      4   1.52
phi[5]              1.09  1.7e-3 3.4e-3   1.08   1.09   1.09   1.09   1.09      4   1.62
phi[6]               1.1  6.8e-4 1.9e-3    1.1    1.1    1.1    1.1    1.1      8   0.99
mu[0]              1.1e4  312.97 625.93 8949.5  1.1e4  1.1e4  1.1e4  1.1e4      4   1.37
mu[1]            

/home/inonpe/jupyter/dtu_bda_project/env/lib/python3.5/site-packages/ipykernel_launcher.py:22: UserWarning: Pickling fit objects is an experimental feature!
The relevant StanModel instance must be pickled along with this fit object.
When unpickling the StanModel must be unpickled first.


2570.4699825629446